In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from IPython import display
display.clear_output()
from fastai.tabular import *
# import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [0]:
df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])

procs = [FillMissing, Categorify, Normalize]
num_cal = df_D.columns.values
cat_col = []

df_D['Y'] = df['Y']

data = (TabularList.from_df(df_D, procs=procs, cont_names=num_cal, cat_names=cat_col)
          .split_by_idx(valid_idx=range(int(len(df_D)*0.9),len(df_D)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(test_df_D, cont_names=num_cal, cat_names=cat_col, procs=procs))
          .databunch())

learn = tabular_learner(data, layers=[50, 50, 20], metrics=accuracy)
learn.fit_one_cycle(20)

epoch,train_loss,valid_loss,accuracy,time
0,0.581710,0.544861,0.818085,00:05
1,0.279262,0.251810,0.903546,00:05
2,0.225035,0.218938,0.902482,00:05
3,0.217797,0.213778,0.905674,00:05
4,0.217547,0.203328,0.909929,00:05
5,0.213263,0.205019,0.904255,00:05
6,0.210589,0.197309,0.906738,00:05
7,0.210225,0.205900,0.907092,00:05
8,0.198638,0.196266,0.907092,00:05
9,0.193794,0.197507,0.908156,00:05


In [0]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')